<a href="https://colab.research.google.com/github/mahfouz72/softmax-and-neural-network-mnist/blob/softmax-model/softmax_and_neural_network_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
# @title Imports
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
import torch
import torch.optim as optim

In [29]:
# @title Global vaiables
learning_rate = 0.01
epochs = 10
batch_size = 64
input_size = 28 * 28
number_of_classes = 10

In [28]:
# @title Data Preparation

transform = transforms.ToTensor()

train_dataset = datasets.MNIST(root='data/MNIST', train=True, transform=transform, download=True)
test_and_validation_dataset = datasets.MNIST(root='data/MNIST', train=False, transform=transform, download=True)

# Extract data and labels for train=False (Split into Validation and Test sets)
test_data = test_and_validation_dataset.data.numpy()
test_labels = test_and_validation_dataset.targets.numpy()

val_data, test_data, val_labels, test_labels = train_test_split(
    test_data, test_labels, test_size=0.5, random_state=123
)

# Wrap NumPy arrays into TensorDataset
val_dataset = TensorDataset(torch.from_numpy(val_data).float(), torch.from_numpy(val_labels).long())
test_dataset = TensorDataset(torch.from_numpy(test_data).float(), torch.from_numpy(test_labels).long())

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

Image batch dimensions: torch.Size([64, 1, 28, 28])
Label batch dimensions: torch.Size([64])
Training set size: 60000 samples
Validation set size: 5000 samples
Test set size: 5000 samples


In [40]:
# @title Softmax Regression Model

class SoftmaxRegression(torch.nn.Module):
    def __init__(self, input_size, number_of_classes):
        super(SoftmaxRegression, self).__init__()
        self.linear = torch.nn.Linear(input_size, number_of_classes)

    def forward(self, x):
      return self.linear(x)

model = SoftmaxRegression(input_size, number_of_classes)
optimizer = optim.SGD(model.parameters(), lr = learning_rate)
loss_function = torch.nn.CrossEntropyLoss()

In [43]:
# @title Training

train_losses = []
val_losses = []
val_accuracies = []

for epoch in range(epochs):
    model.train()  # just setting to training mode
    train_loss = 0.0

    for images, labels in train_loader:
        images = images.view(-1, input_size)

        outputs = model(images)
        loss = loss_function(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    train_losses.append(train_loss / len(train_loader))


    model.eval()
    val_loss = 0.0
    correct_predictions = 0
    total_predictions = 0

    with torch.no_grad():
      for images, labels in val_loader:
        images = images.view(-1, input_size)

        outputs = model(images)
        loss = loss_function(outputs, labels)

        val_loss += loss.item()

        predicition = torch.max(outputs, 1)[1]
        correct_predictions += (predicition == labels).sum().item()
        total_predictions += labels.size(0)

    val_losses.append(val_loss / len(val_loader))
    val_accuracies.append(correct_predictions / total_predictions)


    print(f"Epoch {epoch+1}/{epochs}, "
          f"Train Loss: {train_losses[-1]:.4f}, "
          f"Val Loss: {val_losses[-1]:.4f}, "
          f"Val Accuracy: {val_accuracies[-1] * 100:.4f}%")

Epoch 1/10, Train Loss: 0.3729, Val Loss: 38.7835, Val Accuracy: 90.2600%
Epoch 2/10, Train Loss: 0.3642, Val Loss: 39.4362, Val Accuracy: 90.3200%
Epoch 3/10, Train Loss: 0.3569, Val Loss: 39.8769, Val Accuracy: 90.3200%
Epoch 4/10, Train Loss: 0.3508, Val Loss: 39.5556, Val Accuracy: 90.7000%
Epoch 5/10, Train Loss: 0.3455, Val Loss: 40.0347, Val Accuracy: 90.6800%
Epoch 6/10, Train Loss: 0.3410, Val Loss: 40.5638, Val Accuracy: 90.6400%
Epoch 7/10, Train Loss: 0.3369, Val Loss: 40.8378, Val Accuracy: 90.6400%
Epoch 8/10, Train Loss: 0.3332, Val Loss: 41.2286, Val Accuracy: 90.6200%
Epoch 9/10, Train Loss: 0.3301, Val Loss: 41.9892, Val Accuracy: 90.5600%
Epoch 10/10, Train Loss: 0.3270, Val Loss: 41.2659, Val Accuracy: 90.8600%
